In [ ]:
import pandas as pd
import numpy as np

In [ ]:
raw_data = pd.read_excel('Lego_Raw Data.xlsx')

In [ ]:
def changePos(df, col_name, col_pos):
    cols = list(df.columns)
    cols.insert(col_pos, cols.pop(cols.index(col_name)))
    df = df[cols]
    return df

In [ ]:
def L2_within_L1(raw_data, L1):
    data1 = raw_data.copy()
    data1 = data1.iloc[:, 0:21]
    
    mask1 = (data1['L1 Category'] == L1) | (L1 == "All")
    data = data1[mask1].copy()
    
    data = data.drop_duplicates()
    
    data2 = data.copy()
    data3 = data2.iloc[:, [3,4]].copy()
    data3 = data3.dropna(subset=['L2 Category'])
    data3 = data3.drop_duplicates(subset=['L2 Category']).reset_index(drop=True)
    data3['L2 Position'] = np.nan  # Initialize column to avoid SettingWithCopyWarning
    
    for i in range(data3.shape[0]):
        df1 = data2[data2['L2 Category'] == data3.loc[i, 'L2 Category']].sort_values(by='Date', ascending=False)
        data3.loc[i, 'L2 Position'] = df1.reset_index().loc[0, 'L2 Position']
        
    data = data.assign(Watch_Time=(data['Watch Time'] / 3600))
    
    piv_data = pd.pivot_table(
        data,
        values=['Views', 'Watch_Time', 'Unique Users'],
        index=['L1 Category','L2 Category'],
        aggfunc={'Views': 'sum', 'Watch_Time': 'sum', 'Unique Users':'sum'}
     )
    if(piv_data.shape[0] == 0):
        return piv_data
    
    piv_data = piv_data.assign(awd=np.where(piv_data['Unique Users'] == 0, np.nan, (piv_data['Watch_Time'] / piv_data['Unique Users']) * 60))
    
    piv_data_reset = piv_data.reset_index()
    df = pd.merge(piv_data_reset, data3, on='L2 Category', how='left')
    
    return df

In [ ]:
def CompareL2_L1(df, L1):
    start_date = df['Date'].min()
    end_date = df['Date'].max()

    # Ensure the dates are in the correct format
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Create a loop that runs from start_date to end_date with a 14-day interval
    current_date = start_date

    df_ans = pd.DataFrame()
    flag = True
    i = 1

    # Initialize dictionaries to store last non-null values
    last_non_null = {
        'UU': {},
        'Views': {},
        'WT': {},
        'awd': {}
    }

    while current_date <= end_date:
        print(f"Processing data for the period starting from {current_date.date()}")

        s_date = current_date.strftime('%Y-%m-%d')
        e_date = (current_date + pd.Timedelta(days=14)).strftime('%Y-%m-%d')

        # Add an empty column to df_ans before merging
        df_ans = df_ans.assign(**{f'{s_date}': np.nan})

        mask = df['Date'] >= current_date
        mask1 = df['Date'] < e_date

        df1 = df[mask & mask1].copy()

        df2 = L2_within_L1(df1, L1)

        if df2.shape[0] == 0:
            print("Empty pivot table")
            current_date += pd.Timedelta(days=14)
            continue

        df2.columns = ['L1 Category', 'L2 Category', f'UU{i}', f'Views{i}', f'WT{i}', f'awd{i}', f'L2 Position{i}']

        if flag:
            df_ans = pd.concat([df_ans, df2])
            for col in ['UU', 'Views', 'WT', 'awd']:
                current_col = f'{col}{i}'
                for row_index in df_ans.index:
                    if pd.notna(df_ans.at[row_index, current_col]):
                        # Update the dictionary with the current value
                        last_non_null[col][row_index] = df_ans.at[row_index, current_col]
            flag = False
        else:
            df2.drop(columns=['L1 Category'], inplace=True)
            df_ans = pd.merge(df_ans, df2, on='L2 Category', how='outer')

            for col in ['UU', 'Views', 'WT', 'awd']:
                current_col = f'{col}{i}'
                diff_col = f'Difference_{col}{i}'

                # Initialize the difference column with NaN values
                df_ans[diff_col] = np.nan

                for row_index in df_ans.index:
                    if pd.notna(df_ans.at[row_index, current_col]):
                        # Get the last non-null value for this row
                        if row_index in last_non_null[col]:
                            df_ans.at[row_index, diff_col] = df_ans.at[row_index, current_col] - last_non_null[col][row_index]
                        # Update the dictionary with the current value
                        last_non_null[col][row_index] = df_ans.at[row_index, current_col]

        current_date += pd.Timedelta(days=14)
        i += 1
     
    s_date = current_date.strftime('%Y-%m-%d')
    df_ans = df_ans.assign(**{f'{s_date}': np.nan})
    
    if 'L1 Category' in df_ans.columns:
        # Find the first non-null value in the specified column
        non_null_values = df_ans['L1 Category'].dropna()
        if not non_null_values.empty:
            first_value = non_null_values.iloc[0]
            
            # Replace null values with the first non-null value
            df_ans['L1 Category'] = df_ans['L1 Category'].fillna(first_value)
            
    if 'L1 Category' in df_ans.columns:        
        df_ans = changePos(df_ans, 'L1 Category', 0)
    if 'L2 Category' in df_ans.columns:
        df_ans = changePos(df_ans, 'L2 Category', 1)

    return df_ans

In [ ]:
def highlight_between_dates1(df):
    # Identify the columns with date headers (assumed to be in 'YYYY-MM-DD' format)
    date_columns = [col for col in df.columns if isinstance(col, str) and col.count('-') == 2]

    # Find the ranges between date columns
    ranges = [(df.columns.get_loc(date_columns[i]), df.columns.get_loc(date_columns[i + 1])) for i in range(len(date_columns) - 1)]

    # Create a function to apply borders
    def apply_border(s, left_col, right_col, row_idx, total_rows):
        styles = ['' for _ in s.index]
        if left_col < len(s):
            styles[left_col] = 'border-left: 2px solid black;'
        if right_col - 1 < len(s):
            styles[right_col - 1] = 'border-right: 2px solid black;'
        
        if row_idx == 0:  # First row
            for i in range(left_col, right_col):
                styles[i] += 'border-top: 2px solid black;'
        
        if row_idx == total_rows - 1:  # Last row
            for i in range(left_col, right_col):
                styles[i] += 'border-bottom: 2px solid black;'
        
        return styles

    # Initialize the Styler object
    styler = df.style

    # Apply the border styles between date-indexed columns for first and last row only
    for start, end in ranges:
        styler = styler.apply(apply_border, left_col=start + 1, right_col=end, axis=1, row_idx=0, total_rows=len(df))
        styler = styler.apply(apply_border, left_col=start + 1, right_col=end, axis=1, row_idx=len(df)-1, total_rows=len(df))

    return styler

In [ ]:
import pandas as pd

def highlight_difference_cells(styler):
    # Extract the DataFrame from the Styler object
    df = styler.data
    
    # Identify the columns with headers starting with 'Difference'
    difference_columns = [col for col in df.columns if col.startswith('Difference')]

    # Get the max and min values for scaling the intensity
    max_val = df[difference_columns].max().max()
    min_val = df[difference_columns].min().min()

    # Set a minimum intensity threshold for visibility
    min_intensity = 50

    # Create a function to apply styles based on cell value
    def apply_color(val):
        if val > 0:
            # Scale the intensity and ensure it's at least the minimum threshold
            intensity = max(min_intensity, int(255 * val / max_val))
            green_intensity = intensity
            return 'background-color: green; color : white'
        elif val < 0:
            # Scale the intensity and ensure it's at least the minimum threshold
            intensity = max(min_intensity, int(255 * abs(val) / abs(min_val)))
            red_intensity = intensity
            return 'background-color: red; color : black'
        else:
            return ''

    # Apply the styles to the relevant columns
    styler = styler.applymap(apply_color, subset=difference_columns)

    return styler

In [ ]:
def All_L2_All_L1(df):
    df1 = df['L1 Category'].drop_duplicates().dropna().reset_index().drop(columns=['index']).copy()
    filename = input("Enter the name of file to store the data in .xlsx format: ")
    with pd.ExcelWriter(filename, engine='openpyxl') as writer:
        for index, row in df1.iterrows():
            print(row['L1 Category'])
            pvt = CompareL2_L1(df, row['L1 Category'])
            styled_df = highlight_between_dates1(pvt)
            pvt1 = highlight_difference_cells(styled_df)
            sheet_name = str(row['L1 Category'])
            pvt1.to_excel(writer, sheet_name=sheet_name, index=True)

In [ ]:
df = raw_data.copy()

In [ ]:
All_L2_All_L1(df)